In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
os.environ["TORCH_USE_CUDA_DSA"]= str(0)
device = "cuda"

In [2]:
from __future__ import print_function

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
from torchsummary import summary

c:\Users\admin\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
import import_ipynb
from Datagen_2 import Data

importing Jupyter notebook from Datagen_2.ipynb


In [4]:

seglength = 120
train_ratio = 0.8
focal_train, focal_test, focal_train_labels, focal_test_labels = Data('D:\EEG\FOCAL', train_ratio, seglength, class_num = 0.25)
ige_train, ige_test, ige_train_labels, ige_test_labels = Data('D:\EEG\IGE', train_ratio, seglength, class_num = 0.5)


In [5]:

val_ratio = 0.1
X_train = torch.cat((focal_train, ige_train))
Y_train = torch.cat((focal_train_labels, ige_train_labels))

X_test = torch.cat((focal_train, ige_train))
Y_test = torch.cat((focal_train_labels, ige_train_labels))

X_val = X_train[:int(val_ratio * len(Y_train))]
Y_val = Y_train[:int(val_ratio * len(Y_train))]

X_train = X_train[int(val_ratio * len(Y_train)):]
Y_train = Y_train[int(val_ratio * len(Y_train)):].view(-1, 1)



In [6]:
del focal_train, focal_test, focal_train_labels, focal_test_labels, ige_train, ige_test, ige_train_labels, ige_test_labels

In [7]:
def evaluate(model, X, Y, params = ["acc"]):
    results = []
    
    
    predicted = []
    """ 
    batch_size=4096
    for i in range(int((len(X)/batch_size))):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(X[s:e])
        pred = model(inputs)
        
        predicted.append(pred.data.cpu().numpy())
    """        
        
    inputs = Variable(X)
    predicted = model(inputs)
    
    predicted = predicted.data.cpu().numpy()
    
    for param in params:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted)))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted)))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted))
            recall = recall_score(Y, np.round(predicted))
            results.append(2*precision*recall/ (precision+recall))
    return results

#evaluate(model, X, Y, params = ["acc"])

In [8]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.T = 9000

        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 18))
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        self.padding1 = nn.ZeroPad2d(10)

        # Layer 2
        self.conv2 = nn.Conv2d(16, 320, (5, 5), stride=5)
        self.batchnorm2 = nn.BatchNorm2d(320, False)

        # FC
        self.fc1 = nn.Linear(320*28*4, 1)

    def forward(self, x):
        # Layer 1
        x = F.relu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = self.padding1(x)
        # Layer 2
        x = F.relu(self.conv2(x))
        x = self.batchnorm2(x)


        #x = x.permute(0, 3, 1, 2)



        x = x.reshape(-1, x.shape[1]* x.shape[2]* x.shape[3])
        x = F.sigmoid(self.fc1(x))
        return x



model = CNN().cuda()

#model.forward(X_train[0:200])
#summary(model.cuda(), (1, 120, 20))
#model.cpu().forward(X_train[0:2])

In [9]:
"""
XX_train = torch.randn([565143, 1, 120, 18])
YY_train = torch.randn([565143, 1])
"""

'\nXX_train = torch.randn([565143, 1, 120, 18])\nYY_train = torch.randn([565143, 1])\n'

In [10]:
#np.shape(YY_train)

In [11]:
batch_size = 4096
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)




for epoch in range(3):  # loop over the dataset multiple times
    print("\nEpoch ", epoch)
    running_loss = 0.0
    for i in range(int(len(X_train)/batch_size-1)):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(X_train[s:e].cuda())
        labels = Variable(Y_train[s:e].cuda())



        optimizer.zero_grad()
        outputs = model.forward(inputs)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        #print(loss.data)
    
    


Epoch  0

Epoch  1

Epoch  2


In [12]:
#torch.cuda.empty_cache()
#print(evaluate(model, X_train.cuda(), Y_train.cuda(), params=["fmeasure"]))